In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
print(torch.__version__)
import urllib.request
import torchvision
print(torchvision.__version__)
import torchvision.transforms as transforms
import torch.functional as F
from PIL import Image

1.5.0+cu92
0.6.0+cu92


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
#Loading Model
loaded_model = torch.load('best_model.pt')
loaded_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
transform_test = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

In [5]:
def predict(img1,model,device):
    from PIL import Image
    import torch.nn.functional as F
    
    img1_trans=transform_test(img1)
    img1_trans_reshaped=img1_trans.view(-1,3,224,224)
    out=model(img1_trans_reshaped.to(device))
    out_softmax=F.softmax(out,1)
    max_val,pred=torch.max(out_softmax.data,1)
    classes=['Mask','No Mask']
    prediction=classes[pred.data.item()]
    return(pred.data.item(),prediction)#(result,label)

In [6]:
img_path='test1.png'

predict(Image.open(img_path),loaded_model,device)

(0, 'Mask')

In [7]:
import cv2
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
color_dict={0:(0,255,0),1:(0,0,255)}
url='http://192.168.0.104:4747/video'

In [8]:
def detect_face(img):
    face_img=img.copy()
    face_rects=face_cascade.detectMultiScale(face_img,1.3,5)
    for (x,y,w,h) in face_rects:
        face=img[y:y+h,x:x+w]
        image = Image.fromarray(face,'RGB')
        result,label=predict(image,loaded_model,device) #eg (0,mask)
        cv2.rectangle(face_img,(x,y),(x+w,y+h),color_dict[result],5)
        cv2.putText(face_img, label, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    return face_img

In [9]:
#face detection from video camera
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    frame=detect_face(frame)
    cv2.imshow('Mask Detect',frame)
    k=cv2.waitKey(1)
    if k==27:
        break
    
cap.release()
cv2.destroyAllWindows()
